In [1]:
# Set Variables
## Fixed
tpm_hist = {}  # record tp metric values for trials
tp_metric = 'avgwl'  # specified trade_param_metric: ratio avg value win/loss
## Settable by User
n_trials = 100  # number of HP optimization runs
total_timesteps = 2000 # per HP optimization run
total_episodes = 50
## Logging callback params
lc_threshold=1e-5
lc_patience=15
lc_trial_number=50

In [2]:
# !pip install pyfolio-reloaded  #original pyfolio no longer maintained
# !pip install optuna
# !pip install -U "ray[rllib]"
# !pip install plotly
# !pip install ipywidgets
# !pip install -U kaleido   # enables saving plots to file

In [3]:
! pip install -U kaleido

In [4]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import optuna
from pathlib import Path
from ddpg_torch import Agent
import gym
import numpy as np
import pandas as pd
from utils import sample_data_for_every_nth_day_of_the_month
# from config import *

import config_tickers
from download_data import process_data
from stock_trading_env import StockTradingEnv
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import data_split
from trade_stocks import trade_on_test_df
# from finrl.agents.stablebaselines3.models import DRLAgent

import os
import ray
import kaleido
import itertools

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f'Torch device: {device}')
from plot import get_comparison_df, backtest_stats, backtest_plot, get_daily_return, get_baseline

Torch device: cuda


/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [5]:
import random

In [6]:
# data
# data_path = './data/data_processed_DOW_30_TICKER_2009-01-01_to_2022-07-31.csv'
# processed_full = pd.read_csv(data_path, index_col='Unnamed: 0')

In [7]:
# processed_full.shape

In [8]:
TRAIN_START_DATE = '2009-01-01'

In [9]:
TRAIN_END_DATE = '2016-01-01'

In [10]:
TRADE_START_DATE = '2016-01-01'
TRADE_END_DATE = '2020-05-10'

In [11]:
ticker_name_from_config_tickers = 'DOW_30_TICKER'

TRAIN_CSV_NAME = f'./data/train_{ticker_name_from_config_tickers}_{TRAIN_START_DATE}_to_{TRAIN_END_DATE}.csv'
TRADE_CSV_NAME = f'./data/trade_{ticker_name_from_config_tickers}_{TRADE_START_DATE}_to_{TRADE_END_DATE}.csv'

In [12]:
INDICATORS = [
            'macd',
            'boll_ub',
            'boll_lb',
            'rsi_30',
            'cci_30',
            'dx_30',
            'close_30_sma',
            'close_60_sma'
]


In [13]:
# train = data_split(processed_full, '2009-01-01','2020-07-01')
# trade = data_split(processed_full, '2020-05-01','2021-10-01')

In [14]:
# train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
# trade = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)
# print(f'Number of training samples: {len(train)}')
# print(f'Number of testing samples: {len(trade)}')

In [15]:
print('reading csvs...')
train = pd.read_csv(TRAIN_CSV_NAME, index_col='Unnamed: 0')
trade = pd.read_csv(TRADE_CSV_NAME, index_col='Unnamed: 0')

print(f'Train shape: {train.shape} \n Trade shape: {trade.shape}')

reading csvs...
Train shape: (51098, 18) 
 Trade shape: (31726, 18)


In [16]:
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.771174,746015200.0,4.0,0.003700,2.921925,2.455527,100.000000,66.666667,100.000000,2.688726,2.688726,39.189999,0.000000
0,2009-01-02,AMGN,58.590000,59.080002,57.750000,44.867596,6547900.0,4.0,0.021161,45.729843,43.062191,100.000000,66.666667,100.000000,44.396017,44.396017,39.189999,0.000000
0,2009-01-02,AXP,18.570000,19.520000,18.400000,15.477422,10955700.0,4.0,0.014012,16.048386,14.281916,100.000000,66.666667,100.000000,15.165151,15.165151,39.189999,0.000000
0,2009-01-02,BA,42.799999,45.560001,42.779999,33.941093,7010200.0,4.0,0.043418,35.710279,30.236708,100.000000,66.666667,100.000000,32.973494,32.973494,39.189999,0.000000
0,2009-01-02,CAT,44.910000,46.980000,44.709999,31.942244,7117200.0,4.0,0.034221,33.336670,29.022543,100.000000,66.666667,100.000000,31.179606,31.179606,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761,2015-12-31,UNH,118.639999,119.370003,117.360001,106.529327,2372500.0,3.0,0.783548,108.868949,103.545847,51.120941,71.395955,4.587926,105.130133,105.489747,18.209999,6.217839
1761,2015-12-31,V,78.239998,78.800003,77.550003,74.251747,6538300.0,3.0,-0.037354,77.054661,72.769194,50.625004,-60.912463,11.428758,75.411886,74.513372,18.209999,6.217839
1761,2015-12-31,VZ,46.520000,46.650002,46.209999,34.134270,11490900.0,3.0,0.268690,35.009150,32.901219,52.478840,80.843381,13.649707,33.783726,33.627283,18.209999,6.217839
1761,2015-12-31,WBA,85.919998,86.040001,84.779999,70.057533,2528700.0,3.0,0.508020,72.082260,66.651569,50.439488,83.583229,1.681002,69.086801,69.685235,18.209999,6.217839


In [17]:
train.index

Int64Index([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            ...
            1761, 1761, 1761, 1761, 1761, 1761, 1761, 1761, 1761, 1761],
           dtype='int64', length=51098)

In [18]:
# converting into monthly
train = sample_data_for_every_nth_day_of_the_month(df=train, date='02')
trade = sample_data_for_every_nth_day_of_the_month(df=trade, date='02')
print('Shapes after converting from daily to monthly')
print(f'Train shape: {train.shape} Trade shape: {trade.shape}')

Shapes after converting from daily to monthly
Train shape: (2436, 20) Trade shape: (1537, 20)


In [19]:
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,yyyy_mm_pairs,dd_list
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.771174,746015200.0,4.0,0.003700,2.921925,2.455527,100.000000,66.666667,100.000000,2.688726,2.688726,39.189999,0.000000,2009-01,02
0,2009-01-02,AMGN,58.590000,59.080002,57.750000,44.867596,6547900.0,4.0,0.021161,45.729843,43.062191,100.000000,66.666667,100.000000,44.396017,44.396017,39.189999,0.000000,2009-01,02
0,2009-01-02,AXP,18.570000,19.520000,18.400000,15.477422,10955700.0,4.0,0.014012,16.048386,14.281916,100.000000,66.666667,100.000000,15.165151,15.165151,39.189999,0.000000,2009-01,02
0,2009-01-02,BA,42.799999,45.560001,42.779999,33.941093,7010200.0,4.0,0.043418,35.710279,30.236708,100.000000,66.666667,100.000000,32.973494,32.973494,39.189999,0.000000,2009-01,02
0,2009-01-02,CAT,44.910000,46.980000,44.709999,31.942244,7117200.0,4.0,0.034221,33.336670,29.022543,100.000000,66.666667,100.000000,31.179606,31.179606,39.189999,0.000000,2009-01,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2015-12-02,UNH,116.239998,119.379997,116.110001,106.529327,6071100.0,2.0,-0.425125,106.429345,99.588755,51.662531,92.194243,8.221565,104.058611,105.807105,15.910000,11.309386,2015-12,02
83,2015-12-02,V,79.830002,80.150002,78.970001,75.678398,7310000.0,2.0,0.850311,77.325069,74.405683,56.102777,55.053755,7.974408,75.194852,71.941091,15.910000,11.309386,2015-12,02
83,2015-12-02,VZ,45.450001,45.549999,44.869999,33.174206,12160400.0,2.0,-0.013573,34.125902,32.771220,48.408573,-58.050068,9.996493,33.682794,33.132200,15.910000,11.309386,2015-12,02
83,2015-12-02,WBA,84.040001,84.099998,83.029999,68.535614,2771700.0,2.0,-0.437672,71.260540,65.470906,47.436295,-35.816402,2.398354,69.590253,70.341656,15.910000,11.309386,2015-12,02


In [20]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [21]:
#Defining the environment kwargs

# env_kwargs = {
#     "hmax": 100, 
#     "initial_amount": 1000000, 
#     "buy_cost_pct": 0.001,
#     "sell_cost_pct": 0.001,
#     "state_space": state_space, 
#     "stock_dim": stock_dimension, 
#     "tech_indicator_list": config.INDICATORS, 
#     "action_space": stock_dimension, 
#     "reward_scaling": 1e-4
    
# }
# #Instantiate the training gym compatible environment
# e_train_gym = StockTradingEnv(df = train, **env_kwargs)
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension


env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()

#Instantiate the trading environment
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None, **env_kwargs)

In [22]:
SEED = 333

In [23]:
np.random.seed(SEED)
e_train_gym.seed(SEED)

torch.manual_seed(SEED)
random.seed(SEED)

In [24]:
# agent = DRLAgent(env = env_train)

## Trade performance code

In [25]:
#Main method
# Calculates Trade Performance for Objective
# Called from objective method
# Returns selected trade perf metric(s)
# Requires actions and associated prices

def calc_trade_perf_metric(df_actions, 
                           df_prices_trade,
                           tp_metric,
                           dbg=False):
  
    df_actions_p, df_prices_p, tics = prep_data(df_actions.copy(),
                                                df_prices_trade.copy())
    # actions predicted by trained model on trade data
    df_actions_p.to_csv('df_actions.csv') 

    
    # Confirms that actions, prices and tics are consistent
    df_actions_s, df_prices_s, tics_prtfl = \
        sync_tickers(df_actions_p.copy(),df_prices_p.copy(),tics)
    
    # copy to ensure that tics from portfolio remains unchanged
    tics = tics_prtfl.copy()
    
    # Analysis is performed on each portfolio ticker
    perf_data= collect_performance_data(df_actions_s, df_prices_s, tics)
    # profit/loss for each ticker
    pnl_all = calc_pnl_all(perf_data, tics)
    # values for trade performance metrics
    perf_results = calc_trade_perf(pnl_all)
    df = pd.DataFrame.from_dict(perf_results, orient='index')
    
    # calculate and return trade metric value as objective
    m = calc_trade_metric(df,tp_metric)
    print(f'Ratio Avg Win/Avg Loss: {m}')
    k = str(len(tpm_hist)+1)
    # save metric value
    tpm_hist[k] = m
    return m

In [26]:
# Supporting methods
def calc_trade_metric(df,metric='avgwl'):
    '''# trades', '# wins', '# losses', 'wins total value', 'wins avg value',
       'losses total value', 'losses avg value'''
    # For this tutorial, the only metric available is the ratio of 
    #  average values of winning to losing trades. Others are in development.
    
    # some test cases produce no losing trades.
    # The code below assigns a value as a multiple of the highest value during
    # previous hp optimization runs. If the first run experiences no losses,
    # a fixed value is assigned for the ratio
    tpm_mult = 1.0
    avgwl_no_losses = 25
    if metric == 'avgwl':
        if sum(df['# losses']) == 0:
            try:
                return max(tpm_hist.values())*tpm_mult
            except ValueError:
                return avgwl_no_losses
        avg_w = sum(df['wins total value'])/sum(df['# wins'])
        avg_l = sum(df['losses total value'])/sum(df['# losses'])
        m = abs(avg_w/avg_l)

    return m


def prep_data(df_actions,
              df_prices_trade):
    
    df=df_prices_trade[['date','close','tic']]
    df['Date'] = pd.to_datetime(df['date'])
    df = df.set_index('Date')
    # set indices on both df to datetime
    idx = pd.to_datetime(df_actions.index, infer_datetime_format=True)
    df_actions.index=idx
    tics = np.unique(df.tic)
    n_tics = len(tics)
    print(f'Number of tickers: {n_tics}')
    print(f'Tickers: {tics}')
    dategr = df.groupby('tic')
    p_d={t:dategr.get_group(t).loc[:,'close'] for t in tics}
    df_prices = pd.DataFrame.from_dict(p_d)
    df_prices.index = df_prices.index.normalize()
    return df_actions, df_prices, tics


# prepares for integrating action and price files
def link_prices_actions(df_a,
                        df_p):
    cols_a = [t + '_a' for t in df_a.columns]
    df_a.columns = cols_a
    cols_p = [t + '_p' for t in df_p.columns]
    df_p.columns = cols_p
    return df_a, df_p


def sync_tickers(df_actions,df_tickers_p,tickers):
    # Some DOW30 components may not be included in portfolio
    # passed tickers includes all DOW30 components
    # actions and ticker files may have different length indices
    if len(df_actions) != len(df_tickers_p):
        msng_dates = set(df_actions.index)^set(df_tickers_p.index)
        try:
            #assumption is prices has one additional timestamp (row)
            df_tickers_p.drop(msng_dates,inplace=True)
        except:
            df_actions.drop(msng_dates,inplace=True)
    df_actions, df_tickers_p = link_prices_actions(df_actions,df_tickers_p)
    # identify any DOW components not in portfolio
    t_not_in_a = [t for t in tickers if t + '_a' not in list(df_actions.columns)]
  
    # remove t_not_in_a from df_tickers_p
    drop_cols = [t + '_p' for t in t_not_in_a]
    df_tickers_p.drop(columns=drop_cols,inplace=True)
    
    # Tickers in portfolio
    tickers_prtfl = [c.split('_')[0] for c in df_actions.columns]
    return df_actions,df_tickers_p, tickers_prtfl

def collect_performance_data(dfa,dfp,tics, dbg=False):
    
    perf_data = {}
    # In current version, files columns include secondary identifier
    for t in tics:
        # actions: purchase/sale of DOW equities
        acts = dfa['_'.join([t,'a'])].values
        # ticker prices
        prices = dfp['_'.join([t,'p'])].values
        # market value of purchases/sales
        tvals_init = np.multiply(acts,prices)
        d={'actions':acts, 'prices':prices,'init_values':tvals_init}
        perf_data[t]=d

    return perf_data


def calc_pnl_all(perf_dict, tics_all):
    # calculate profit/loss for each ticker
    print(f'Calculating profit/loss for each ticker')
    pnl_all = {}
    for tic in tics_all:
        pnl_t = []
        tic_data = perf_dict[tic]
        init_values = tic_data['init_values']
        acts = tic_data['actions']
        prices = tic_data['prices']
        cs = np.cumsum(acts)
        args_s = [i + 1 for i in range(len(cs) - 1) if cs[i + 1] < cs[i]]
        # tic actions with no sales
        if not args_s:
            pnl = complete_calc_buyonly(acts, prices, init_values)
            pnl_all[tic] = pnl
            continue
        # copy acts: acts_rev will be revised based on closing/reducing init positions
        pnl_all = execute_position_sales(tic,acts,prices,args_s,pnl_all)

    return pnl_all


def complete_calc_buyonly(actions, prices, init_values):
    # calculate final pnl for each ticker assuming no sales
    fnl_price = prices[-1]
    final_values = np.multiply(fnl_price, actions)
    pnl = np.subtract(final_values, init_values)
    return pnl


def execute_position_sales(tic,acts,prices,args_s,pnl_all):
  # calculate final pnl for each ticker with sales
    pnl_t = []
    acts_rev = acts.copy()
    # location of sales transactions
    for s in args_s:  # s is scaler
        # price_s = [prices[s]]
        act_s = [acts_rev[s]]
        args_b = [i for i in range(s) if acts_rev[i] > 0]
        prcs_init_trades = prices[args_b]
        acts_init_trades = acts_rev[args_b]
  
        # update actions for sales
        # reduce/eliminate init values through trades
        # always start with earliest purchase that has not been closed through sale
        # selectors for purchase and sales trades
        # find earliest remaining purchase
        arg_sel = min(args_b)
        # sel_s = len(acts_trades) - 1

        # closing part/all of earliest init trade not yet closed
        # sales actions are negative
        # in this test case, abs_val of init and sales share counts are same
        # zero-out sales actions
        # market value of sale
        # max number of shares to be closed: may be less than # originally purchased
        acts_shares = min(abs(act_s.pop()), acts_rev[arg_sel])

        # mv of shares when purchased
        mv_p = abs(acts_shares * prices[arg_sel])
        # mv of sold shares
        mv_s = abs(acts_shares * prices[s])

        # calc pnl
        pnl = mv_s - mv_p
        # reduce init share count
        # close all/part of init purchase
        acts_rev[arg_sel] -= acts_shares
        acts_rev[s] += acts_shares
        # calculate pnl for trade
        # value of associated purchase
        
        # find earliest non-zero positive act in acts_revs
        pnl_t.append(pnl)
    
    pnl_op = calc_pnl_for_open_positions(acts_rev, prices)
    #pnl_op is list
    # add pnl_op results (if any) to pnl_t (both lists)
    pnl_t.extend(pnl_op)
    #print(f'Total pnl for {tic}: {np.sum(pnl_t)}')
    pnl_all[tic] = np.array(pnl_t)
    return pnl_all


def calc_pnl_for_open_positions(acts,prices):
    # identify any positive share values after accounting for sales
    pnl = []
    fp = prices[-1] # last price
    open_pos_arg = np.argwhere(acts>0)
    if len(open_pos_arg)==0:
        return pnl # no open positions

    mkt_vals_open = np.multiply(acts[open_pos_arg], prices[open_pos_arg])
    # mkt val at end of testing period
    # treat as trades for purposes of calculating pnl at end of testing period
    mkt_vals_final = np.multiply(fp, acts[open_pos_arg])
    pnl_a = np.subtract(mkt_vals_final, mkt_vals_open)
    #convert to list
    pnl = [i[0] for i in pnl_a.tolist()]
    #print(f'Market value of open positions at end of testing {pnl}')
    return pnl


def calc_trade_perf(pnl_d):
    # calculate trade performance metrics
    perf_results = {}
    for t,pnl in pnl_d.items():
        wins = pnl[pnl>0]  # total val
        losses = pnl[pnl<0]
        n_wins = len(wins)
        n_losses = len(losses)
        n_trades = n_wins + n_losses
        wins_val = np.sum(wins)
        losses_val = np.sum(losses)
        wins_avg = 0 if n_wins==0 else np.mean(wins)
        #print(f'{t} n_wins: {n_wins} n_losses: {n_losses}')
        losses_avg = 0 if n_losses==0 else np.mean(losses)
        d = {'# trades':n_trades,'# wins':n_wins,'# losses':n_losses,
             'wins total value':wins_val, 'wins avg value':wins_avg,
             'losses total value':losses_val, 'losses avg value':losses_avg,}
        perf_results[t] = d
    return perf_results

## Tuning using Optuna}

In [27]:
# def sample_ddpg_params_all(trial:optuna.Trial,
#                            # fixed values from previous study
#                            learning_rate=0.0103,
#                            batch_size=128,
#                            buffer_size=int(1e6)):

#     gamma = trial.suggest_categorical("gamma", [0.94, 0.96, 0.98])
#     # Polyak coeff
#     tau = trial.suggest_categorical("tau", [0.08, 0.1, 0.12])

#     train_freq = trial.suggest_categorical("train_freq", [512,768,1024])
#     gradient_steps = train_freq
    
#     noise_type = trial.suggest_categorical("noise_type", ["ornstein-uhlenbeck", "normal", None])
#     noise_std = trial.suggest_categorical("noise_std", [.1,.2,.3] )

#     # NOTE: Add "verybig" to net_arch when tuning HER (see TD3)
#     net_arch = trial.suggest_categorical("net_arch", ["small", "big"])
#     # activation_fn = trial.suggest_categorical('activation_fn', [nn.Tanh, nn.ReLU, nn.ELU, nn.LeakyReLU])

#     net_arch = {
#         "small": [64, 64],
#         "medium": [256, 256],
#         "big": [512, 512],
#     }[net_arch]
  
#     hyperparams = {
#         "batch_size": batch_size,
#         "buffer_size": buffer_size,
#         "gamma": gamma,
#         "gradient_steps": gradient_steps,
#         "learning_rate": learning_rate,
#         "tau": tau,
#         "train_freq": train_freq,
#         #"noise_std": noise_std,
#         #"noise_type": noise_type,
        
#         "policy_kwargs": dict(net_arch=net_arch)
#     }
#     return hyperparams

In [28]:
1-0.2

0.8

In [29]:
def sample_ddpg_params(trial:optuna.Trial):
    # Size of the replay buffer
    buffer_size = trial.suggest_categorical("buffer_size", [int(1e3), int(1e5), int(1e6)])
#     tau = trial.suggest_categorical("tau", [0.2, 0.3, 0.01, 0.001])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
#     learning_rate_critic = trial.suggest_loguniform("learning_rate_critic", 1e-5, 1e-1)
#     learning_rate_actor = 10**trial.suggest_int('logval', -5, 0)
#     learning_rate_critic = 10**trial.suggest_int('logval', -5, 0)
    
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16, 32, 64])

    net_arch = trial.suggest_categorical("net_arch", ["small", "medium", "big"])

    net_arch = {
        "small": [64, 64],
        "medium": [256, 256],
        "default": [400, 400],
        "big": [512, 512],
    }[net_arch]

    return {
#             ,
#             
#             "learning_rate_actor": learning_rate_actor,
#             "tau": tau,
            "learning_rate": learning_rate,
            "buffer_size": buffer_size,
            "batch_size": batch_size,
            "layer_1_size": net_arch[0],
            "layer_2_size": net_arch[1]}

In [30]:
class LoggingCallback:
    def __init__(self, threshold, trial_number, patience):
        '''
        threshold:int tolerance for increase in objective
        trial_number: int Prune after minimum number of trials
        patience: int patience for the threshold
        '''
        self.threshold = threshold
        self.trial_number  = trial_number
        self.patience = patience
        print(f'Callback threshold {self.threshold}, \
            trial_number {self.trial_number}, \
            patience {self.patience}')
        self.cb_list = [] #Trials list for which threshold is reached
        
    def __call__(self, study:optuna.study, frozen_trial:optuna.Trial):
        #Setting the best value in the current trial
        study.set_user_attr("previous_best_value", study.best_value)

        #Checking if the minimum number of trials have pass
        if frozen_trial.number >self.trial_number:
            previous_best_value = study.user_attrs.get("previous_best_value",None)
            #Checking if the previous and current objective values have the same sign
            if previous_best_value * study.best_value >=0:
                #Checking for the threshold condition
                if abs(previous_best_value-study.best_value) < self.threshold: 
                    self.cb_list.append(frozen_trial.number)
                    #If threshold is achieved for the patience amount of time
                    if len(self.cb_list)>self.patience:
                        print('The study stops now...')
                        print('With number',frozen_trial.number ,'and value ',frozen_trial.value)
                        print('The previous and current best values are {} and {} respectively'
                              .format(previous_best_value, study.best_value))
                        study.stop()

In [31]:
from IPython.display import clear_output
import sys   

os.makedirs("./models_monthly", exist_ok=True)

def objective(trial:optuna.Trial):
    #Trial will suggest a set of hyperparamters from the specified range

    # Optional to optimize larger set of parameters
    # hyperparameters = sample_ddpg_params_all(trial)

    # Optimize buffer size, batch size, learning rate
    hyperparameters = sample_ddpg_params(trial)
    print(f'Hyperparameters from objective: {hyperparameters}')
    
#     policy_kwargs = None  # default
#     if 'policy_kwargs' in hyperparameters.keys():
#         policy_kwargs = hyperparameters['policy_kwargs']
#         del hyperparameters['policy_kwargs']
    #print(f'Policy keyword arguments {policy_kwargs}')
#     model_ddpg = agent.get_model("ddpg",
#                                policy_kwargs = policy_kwargs,
#                                model_kwargs = hyperparameters )
#     #You can increase it for better comparison
#     trained_ddpg = agent.train_model(model=model_ddpg,
#                                    tb_log_name="ddpg",
#                                    total_timesteps=total_timesteps)
    ACTOR_LR = hyperparameters["learning_rate"]
    CRITIC_LR = hyperparameters["learning_rate"]
    BATCH_SIZE = hyperparameters["batch_size"]
    TAU = 0.001
    LAYER_1_SIZE = hyperparameters["layer_1_size"]
    LAYER_2_SIZE = hyperparameters["layer_2_size"]
    buffer_size = hyperparameters["buffer_size"]
    
    model_ddpg = Agent(alpha=ACTOR_LR, beta=CRITIC_LR, ckp_dir='./models_monthly', input_dims=state_space, tau=TAU,
              batch_size=BATCH_SIZE, layer1_size=LAYER_1_SIZE, layer2_size=LAYER_2_SIZE, max_size=buffer_size,
              n_actions=stock_dimension)
    
    trained_ddpg = model_ddpg.train_model(
                        total_episodes=total_episodes, train_from_scratch=True, 
                        env=e_train_gym, env_kwargs=env_kwargs, save_ckp=False, ckp_save_freq=0,
                        use_wandb=False)
    
#     trained_ddpg.save('models/ddpg_{}.pth'.format(trial.number))
    trained_ddpg.save_checkpoint(checkpoint_name='ddpg_{}.pt'.format(trial.number), last_episode=trained_ddpg.episode)
    
    clear_output(wait=True)
    #For the given hyperparamters, determine the account value in the trading period
    
#     df_account_value, df_actions = DRLAgent.DRL_prediction(
#     model=trained_ddpg, 
#     environment = e_trade_gym)
    df_account_value, df_actions, cumulative_rewards_test = trade_on_test_df(df=trade, 
                                                                             model=trained_ddpg, 
                                                                             train_df=train, 
                                                                             env_kwargs=env_kwargs)

    # Calculate trade performance metric
    # Currently ratio of average win and loss market values
    tpm = calc_trade_perf_metric(df_actions,trade,tp_metric)
    return tpm

#Create a study object and specify the direction as 'maximize'
#As you want to maximize sharpe
#Pruner stops not promising iterations
#Use a pruner, else you will get error related to divergence of model
#You can also use Multivariate samplere
#sampler = optuna.samplers.TPESampler(multivarite=True,seed=42)
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name="ddpg_study_monthly",direction='maximize',
                        sampler = sampler, pruner=optuna.pruners.HyperbandPruner())

logging_callback = LoggingCallback(threshold=lc_threshold,
                               patience=lc_patience,
                               trial_number=lc_trial_number)
#You can increase the n_trials for a better search space scanning
study.optimize(objective, n_trials=n_trials, catch=(ValueError,), callbacks=[logging_callback])

[I 2022-07-29 14:23:00,706] Trial 46 finished with value: 1.0995206534627102 and parameters: {'buffer_size': 1000, 'learning_rate': 0.0006195777142295596, 'batch_size': 4, 'net_arch': 'small'}. Best is trial 37 with value: 1.6970930732782818.


hit end!
Number of tickers: 29
Tickers: ['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV'
 'UNH' 'V' 'VZ' 'WBA' 'WMT']
Calculating profit/loss for each ticker
Ratio Avg Win/Avg Loss: 1.0995206534627102
Hyperparameters from objective: {'learning_rate': 1.0669237019941647e-05, 'buffer_size': 1000000, 'batch_size': 32, 'layer_1_size': 512, 'layer_2_size': 512}
day: 83, episode: 2352
begin_total_asset: 1000000.00
end_total_asset: 1187426.00
total_reward: 187426.00
total_cost: 592.22
total_trades: 1381
Sharpe: 3.937
day: 83, episode: 2354
begin_total_asset: 1000000.00
end_total_asset: 2038894.70
total_reward: 1038894.70
total_cost: 998.99
total_trades: 1577
Sharpe: 4.701
day: 83, episode: 2356
begin_total_asset: 1000000.00
end_total_asset: 2302361.10
total_reward: 1302361.10
total_cost: 999.00
total_trades: 1577
Sharpe: 4.878
day: 83, episode: 2358
begin_total_asset: 1000000.00
end_total_asset: 2

[W 2022-07-29 14:24:09,665] Trial 47 failed because of the following error: RuntimeError('[enforce fail at inline_container.cc:319] . unexpected pos 128 vs 0')
Traceback (most recent call last):
  File "/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/torch/serialization.py", line 379, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/torch/serialization.py", line 591, in _save
    zip_file.write_record('data.pkl', data_value, len(data_value))
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_11562/2773307561.py", line 46, in objective
    trained_ddpg.save_checkpoint(checkpoint_name='ddpg_

RuntimeError: [enforce fail at inline_container.cc:319] . unexpected pos 128 vs 0

In [32]:
!ls models_monthly

ddpg_0.pt   ddpg_17.pt	ddpg_24.pt  ddpg_31.pt	ddpg_39.pt  ddpg_46.pt
ddpg_10.pt  ddpg_18.pt	ddpg_25.pt  ddpg_32.pt	ddpg_3.pt   ddpg_47.pt
ddpg_11.pt  ddpg_19.pt	ddpg_26.pt  ddpg_33.pt	ddpg_40.pt  ddpg_4.pt
ddpg_12.pt  ddpg_1.pt	ddpg_27.pt  ddpg_34.pt	ddpg_41.pt  ddpg_5.pt
ddpg_13.pt  ddpg_20.pt	ddpg_28.pt  ddpg_35.pt	ddpg_42.pt  ddpg_6.pt
ddpg_14.pt  ddpg_21.pt	ddpg_29.pt  ddpg_36.pt	ddpg_43.pt  ddpg_7.pt
ddpg_15.pt  ddpg_22.pt	ddpg_2.pt   ddpg_37.pt	ddpg_44.pt  ddpg_8.pt
ddpg_16.pt  ddpg_23.pt	ddpg_30.pt  ddpg_38.pt	ddpg_45.pt  ddpg_9.pt


In [33]:
study.best_trial

FrozenTrial(number=37, values=[1.6970930732782818], datetime_start=datetime.datetime(2022, 7, 29, 14, 13, 26, 484693), datetime_complete=datetime.datetime(2022, 7, 29, 14, 14, 28, 410162), params={'buffer_size': 1000000, 'learning_rate': 0.0031060995478059458, 'batch_size': 8, 'net_arch': 'big'}, distributions={'buffer_size': CategoricalDistribution(choices=(1000, 100000, 1000000)), 'learning_rate': LogUniformDistribution(high=0.1, low=1e-05), 'batch_size': CategoricalDistribution(choices=(4, 8, 16, 32, 64)), 'net_arch': CategoricalDistribution(choices=('small', 'medium', 'big'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=37, state=TrialState.COMPLETE, value=None)

In [ ]:
study.trials[5]

In [ ]:
cccc

In [ ]:
import joblib

In [48]:
SEED

333

In [ ]:
joblib.dump(study, "final_ddpg_study_a_c_same_monthly_seed_12321_.pkl")

In [ ]:
# from finrl import config

In [34]:
#Get the best hyperparamters
print('Hyperparameters after tuning',study.best_params)
# print('Hyperparameters before tuning',config.DDPG_PARAMS)

Hyperparameters after tuning {'buffer_size': 1000000, 'learning_rate': 0.0031060995478059458, 'batch_size': 8, 'net_arch': 'big'}


In [ ]:
study.best_trial

In [ ]:
# from stable_baselines3 import DDPG
# tuned_model_ddpg = DDPG.load('models/ddpg_{}.pth'.format(study.best_trial.number),env=env_train)

In [35]:
# from config import *

agent = Agent(alpha= 0.0031060995478059458, beta=0.0031060995478059458, ckp_dir='', input_dims=state_space, tau=0.001,
              batch_size=8, layer1_size=512, layer2_size=512, max_size=1000000,
              n_actions=stock_dimension)
            
_ = agent.load_checkpoint('./models_monthly/ddpg_{}.pt'.format(37))

Loading checkpoint...(./models_monthly/ddpg_37.pt)
Loaded model at episode 50 from ./models_monthly/ddpg_37.pt


In [ ]:
_

In [ ]:
#Trading period account value with tuned model
# df_account_value_tuned, df_actions_tuned = DRLAgent.DRL_prediction(
#     model=tuned_model_ddpg, 
#     environment = e_trade_gym)

In [36]:
trade

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,yyyy_mm_pairs,dd_list
0,2016-01-04,AAPL,25.652500,26.342501,25.500000,24.185041,270597600.0,0.0,-0.607122,27.514333,23.324075,40.592665,-128.001124,46.712235,25.958001,26.239099,20.700001,32.448007,2016-01,04
0,2016-01-04,AMGN,159.000000,159.220001,156.089996,131.357422,5083200.0,0.0,0.701384,137.610728,129.128536,49.850159,-99.000825,20.083907,133.444707,131.115378,20.700001,32.448007,2016-01,04
0,2016-01-04,AXP,68.089996,68.180000,66.769997,61.000008,9248300.0,0.0,-0.619022,64.551295,61.128187,39.427827,-181.827561,36.141994,63.493901,65.354404,20.700001,32.448007,2016-01,04
0,2016-01-04,BA,141.380005,141.699997,139.429993,126.005112,5719500.0,0.0,-0.410402,134.397851,125.168170,46.774696,-168.096150,17.581052,130.637844,129.631527,20.700001,32.448007,2016-01,04
0,2016-01-04,CAT,66.879997,68.080002,65.720001,56.383404,8586900.0,0.0,-0.264871,58.655484,53.176711,46.887845,-46.942373,20.469329,56.936261,57.966606,20.700001,32.448007,2016-01,04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,2020-05-04,UNH,284.489990,287.940002,282.109985,278.351166,2894200.0,0.0,8.574687,297.964279,241.961979,53.502956,67.414550,17.766145,255.963606,259.047151,35.970001,29.753794,2020-05,04
52,2020-05-04,V,172.949997,176.940002,171.720001,173.640854,9386200.0,0.0,2.110878,178.857049,157.134771,50.746353,91.747861,4.325559,163.635423,173.513909,35.970001,29.753794,2020-05,04
52,2020-05-04,VZ,56.849998,57.029999,55.799999,50.752518,11833600.0,0.0,0.532658,53.052811,50.518367,49.770764,16.704936,12.646976,50.235072,50.278993,35.970001,29.753794,2020-05,04
52,2020-05-04,WBA,41.700001,42.820000,41.220001,38.652817,7189100.0,0.0,-0.546531,41.692181,37.704073,45.066373,-95.612294,11.501582,39.666865,42.244663,35.970001,29.753794,2020-05,04


In [37]:
df_account_value_tuned, df_actions_tuned, cum_rewards_test = trade_on_test_df(df=trade, model=agent, train_df=train, env_kwargs=env_kwargs)

hit end!


In [38]:
df_account_value_tuned

,date,account_value
0,2016-01-04,1.000000e+06
1,2016-02-02,9.962295e+05
2,2016-03-02,1.003217e+06
3,2016-04-04,1.018370e+06
4,2016-05-02,1.019710e+06
5,2016-06-02,1.027711e+06
6,2016-07-05,1.029542e+06
7,2016-08-02,1.037101e+06
8,2016-09-02,1.034208e+06
9,2016-10-03,1.024383e+06


In [39]:
def add_trade_perf_metric(df_actions, 
                          perf_stats_all,
                          trade, 
                          tp_metric):
    tpm = calc_trade_perf_metric(df_actions,trade,tp_metric)
    trp_metric = {'Value':tpm}
    df2 = pd.DataFrame(trp_metric,index=['Trade_Perf'])
    perf_stats_all = perf_stats_all.append(df2)
    return perf_stats_all

In [40]:
os.makedirs('./tuning_monthly', exist_ok=True)

In [41]:
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')
df_actions_tuned.to_csv("./"+"tuning_monthly"+"/tuned_actions_" +now+ '.csv')

In [42]:
now

'20220729-14h39'

In [43]:
#Backtesting with our pruned model
print("==============Get Backtest Results===========")
print("==============Pruned Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all_tuned = backtest_stats(account_value=df_account_value_tuned)
perf_stats_all_tuned = pd.DataFrame(perf_stats_all_tuned)
perf_stats_all_tuned.columns = ['Value']
# add trade performance metric
perf_stats_all_tuned = \
  add_trade_perf_metric(df_actions_tuned,
                        perf_stats_all_tuned,
                        trade,
                        tp_metric)
perf_stats_all_tuned.to_csv("./"+"tuning_monthly"+"/perf_stats_all_tuned_"+now+'.csv')

==============Get Backtest Results===========
==============Pruned Model===========
Annual return           6.840064
Cumulative returns      0.542024
Annual volatility       0.393262
Sharpe ratio            5.551782
Calmar ratio           85.654155
Stability               0.926632
Max drawdown           -0.079857
Omega ratio             3.048536
Sortino ratio           9.844868
Skew                         NaN
Kurtosis                     NaN
Tail ratio              2.252221
Daily value at risk    -0.040882
dtype: float64
Number of tickers: 29
Tickers: ['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV'
 'UNH' 'V' 'VZ' 'WBA' 'WMT']
Calculating profit/loss for each ticker
Ratio Avg Win/Avg Loss: 1.7784658769694537


/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/tmp/ipykernel_11562/490943077.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  perf_stats_all = perf_stats_all.append(df2)


In [44]:
from config import BASELINE_TICKER_NAME_BACKTESTING

In [45]:
# DJI for the same time
results_df = get_comparison_df(df_account_value_tuned, BASELINE_TICKER_NAME_BACKTESTING)
train_values = np.zeros(len(results_df))
train_values[list(results_df.metric).index('Cumulative returns')] = cum_rewards_test[-1]
train_values[list(results_df.metric).index('Max drawdown')] = min(cum_rewards_test)
results_df['train_data'] = train_values

/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


Annual return           6.840064
Cumulative returns      0.542024
Annual volatility       0.393262
Sharpe ratio            5.551782
Calmar ratio           85.654155
Stability               0.926632
Max drawdown           -0.079857
Omega ratio             3.048536
Sortino ratio           9.844868
Skew                         NaN
Kurtosis                     NaN
Tail ratio              2.252221
Daily value at risk    -0.040882
dtype: float64
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1090, 8)
Annual return          0.077917
Cumulative returns     0.383391
Annual volatility      0.200716
Sharpe ratio           0.475233
Calmar ratio           0.210098
Stability              0.800584
Max drawdown          -0.370862
Omega ratio            1.115106
Sortino ratio          0.650485
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.816796
Daily value at risk   -0.024909
dtype: float64


/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/finrl/meta/preprocessor/yahoodownloader.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [46]:
# SAVING THIS DF_ACCOUNT_VALUE_TUNED

account_value_csv_name = f'account_value_test_tuning.csv'
actions_csv_name = f'daily_actions_test_tuning.csv'
results_table_name = f'return_comparison_test.csv'
df_account_value_tuned.to_csv(os.path.join('./tuning_monthly', account_value_csv_name))
df_actions_tuned.to_csv(os.path.join('./tuning_monthly', actions_csv_name))
results_df.to_csv(os.path.join('./tuning_monthly', results_table_name))

In [47]:
SEED

333

In [ ]:
#Now train with not tuned hyperaparameters
#Default config.ddpg_PARAMS
non_tuned_model_ddpg = agent.get_model("ddpg",model_kwargs = config.DDPG_PARAMS )
trained_ddpg = agent.train_model(model=non_tuned_model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=total_timesteps)

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

In [ ]:
#Backtesting for not tuned hyperparamters
print("==============Get Backtest Results===========")
print("============Default Hyperparameters==========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.columns = ['Value']
# add trade performance metric
perf_stats_all = add_trade_perf_metric(df_actions,
                        perf_stats_all,
                        trade,
                        tp_metric)
perf_stats_all.to_csv("./"+"tuning"+"/perf_stats_all_untuned_"+now+'.csv')

In [ ]:
study = joblib.load("final_ddpg_study__.pkl")

In [ ]:
#Certainly you can afford more number of trials for further optimization
from optuna.visualization import plot_optimization_history
fig = plot_optimization_history(study)
#"./"+config.RESULTS_DIR+
fig.write_image("./"+"tuning"+"/opt_hist.png")
fig.show()

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
#Hyperparamters importance

try:
    fig = plot_param_importances(study)
    fig.write_image("./"+config.RESULTS_DIR+"/params_importances.png")
    fig.show()
except:
    print('Cannot calculate hyperparameter importances: no variation')

In [ ]:
# fig = plot_edf(study)
# fig.write_image("./"+"tuning"+"/emp_dist_func.png")
# fig.show()